In [8]:
import pandas as pd
import glob
import os

# Find all storedata CSV files
csv_files = glob.glob("storedata*.csv")
print(f"Found {len(csv_files)} storedata CSV files:")
for file in csv_files:
    print(f"  - {file}")

# Read all CSV files and check columns
dataframes = []
all_columns = []

print("\n" + "="*80)
print("Checking columns in each file:")
print("="*80)

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    all_columns.append(set(df.columns))
    print(f"\n{file}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")

# Check if all files have the same columns
print("\n" + "="*80)
print("Column Comparison:")
print("="*80)

if all(cols == all_columns[0] for cols in all_columns):
    print("✓ All files have the SAME columns!")
else:
    print("✗ Files have DIFFERENT columns!")
    # Show differences
    for i, (file, cols) in enumerate(zip(csv_files, all_columns)):
        if i > 0:
            diff = cols.symmetric_difference(all_columns[0])
            if diff:
                print(f"\n{file} differences: {diff}")

# Concatenate all dataframes
print("\n" + "="*80)
print("Concatenating all files:")
print("="*80)

combined_df = pd.concat(dataframes, ignore_index=True)
print(f"\nTotal rows BEFORE removing duplicates: {len(combined_df)}")
print(f"Shape: {combined_df.shape}")
print(f"\nFirst 5 rows:")
print(combined_df.head())

# Remove duplicate rows (keep unique values)
unique_df = combined_df.drop_duplicates()

print("\n" + "="*80)
print("After removing duplicates:")
print("="*80)
print(f"\nTotal rows AFTER removing duplicates: {len(unique_df)}")
print(f"Shape: {unique_df.shape}")
print(f"\nRows removed: {len(combined_df) - len(unique_df)}")

print(f"\nFirst 5 rows of unique data:")
print(unique_df.head())

print(f"\nLast 5 rows of unique data:")
print(unique_df.tail())

# Display summary statistics
print("\n" + "="*80)
print("Summary:")
print("="*80)
print(f"Number of files processed: {len(csv_files)}")
print(f"Total rows before deduplication: {len(combined_df)}")
print(f"Total rows after deduplication: {len(unique_df)}")
print(f"Duplicate rows removed: {len(combined_df) - len(unique_df)}")
print(f"Number of columns: {len(unique_df.columns)}")


Found 7 storedata CSV files:
  - storedata (1).csv
  - storedata (2).csv
  - storedata (3).csv
  - storedata (4).csv
  - storedata (5).csv
  - storedata (6).csv
  - storedata (7).csv

Checking columns in each file:

storedata (1).csv:
  Shape: (33, 34)
  Columns: ['Shop name', 'Shop ID', 'Shop ID.1', 'Order Count', 'Count of misc payment', 'Sales (excl. tax)', 'Tax on sales', 'Sales (incl. tax)', 'Order error adjustments', 'Tax on order error adjustments', 'Order error adjustments (incl. tax)', 'Price adjustments (excl. tax)', 'Tax on price adjustments', 'Price adjustments (incl. tax)', 'Offers on items (incl. tax)', 'Tax on offers on items', 'Delivery offer redemptions (incl. tax)', 'Tax on delivery offer redemptions', 'Offer Redemption Fee', 'Marketing adjustment', 'Bag fee', 'Marketplace Fee', 'Tax on Marketplace Fee', 'Delivery network fee', 'Tax on delivery network fee', 'Order processing fee', 'Total sales after adjustments (incl. tax)', 'Capital payments', 'Other payments', 'Mar

In [12]:
# Convert Payout date to datetime format
unique_df['Payout date'] = pd.to_datetime(unique_df['Payout date'], format='%d/%m/%Y')

print("Date range in the data:")
print(f"Earliest date: {unique_df['Payout date'].min()}")
print(f"Latest date: {unique_df['Payout date'].max()}")

# Filter for Pre Period: 30/6/2025 to 4/8/2025
pre_period_start = pd.to_datetime('30/06/2025', format='%d/%m/%Y')
pre_period_end = pd.to_datetime('04/08/2025', format='%d/%m/%Y')

pre_period_df = unique_df[(unique_df['Payout date'] >= pre_period_start) & 
                           (unique_df['Payout date'] <= pre_period_end)]

print("\n" + "="*80)
print("PRE PERIOD: 30/6/2025 to 4/8/2025")
print("="*80)
print(f"Number of rows in pre period: {len(pre_period_df)}")

# Create pivot table for pre period
pre_pivot = pre_period_df.groupby('Shop name').agg({
    'Sales (excl. tax)': 'sum',
    'Total payout ': 'sum'
}).round(2)

pre_pivot.columns = ['Sales (excl. tax) - Pre', 'Total Payout - Pre']
print("\nPre Period Pivot Table:")
print(pre_pivot)

# Filter for Post Period: 11/8/2025 to 29/8/2025
post_period_start = pd.to_datetime('11/08/2025', format='%d/%m/%Y')
post_period_end = pd.to_datetime('29/08/2025', format='%d/%m/%Y')

post_period_df = unique_df[(unique_df['Payout date'] >= post_period_start) & 
                            (unique_df['Payout date'] <= post_period_end)]

print("\n" + "="*80)
print("POST PERIOD: 11/8/2025 to 29/8/2025")
print("="*80)
print(f"Number of rows in post period: {len(post_period_df)}")

# Create pivot table for post period
post_pivot = post_period_df.groupby('Shop name').agg({
    'Sales (excl. tax)': 'sum',
    'Total payout ': 'sum'
}).round(2)

post_pivot.columns = ['Sales (excl. tax) - Post', 'Total Payout - Post']
print("\nPost Period Pivot Table:")
print(post_pivot)

# Summary comparison
print("\n" + "="*80)
print("SUMMARY COMPARISON")
print("="*80)
print(f"\nPre Period (30/6/2025 - 4/8/2025):")
print(f"  Total Sales (excl. tax): ${pre_pivot['Sales (excl. tax) - Pre'].sum():,.2f}")
print(f"  Total Payout: ${pre_pivot['Total Payout - Pre'].sum():,.2f}")
print(f"  Number of stores: {len(pre_pivot)}")

print(f"\nPost Period (11/8/2025 - 29/8/2025):")
print(f"  Total Sales (excl. tax): ${post_pivot['Sales (excl. tax) - Post'].sum():,.2f}")
print(f"  Total Payout: ${post_pivot['Total Payout - Post'].sum():,.2f}")
print(f"  Number of stores: {len(post_pivot)}")


Date range in the data:
Earliest date: 2025-06-30 00:00:00
Latest date: 2025-09-29 00:00:00

PRE PERIOD: 30/6/2025 to 4/8/2025
Number of rows in pre period: 66

Pre Period Pivot Table:
                                    Sales (excl. tax) - Pre  \
Shop name                                                     
McDonald's® (2310 W. COMMONWEALTH)                 21152.95   
McDonald's® (Glendora)                             12966.53   
McDonald's® (L.A. - 50Th/Figueroa)                 35985.91   
McDonald's® (L.A. - Huntington)                    32351.39   
McDonald's® (Pasadena)                             33410.09   
McDonald's® (Pasadena/Lake)                        47397.68   
McDonald's® (Rosemd & Duarte)                      24000.51   
McDonald's® (S Pasadena-F Oak)                     18652.70   
McDonald's® (San Gabrl/San Ga)                      7375.07   
McDonald's® (Valley/Walnut Grove)                  21099.50   
McDonald's® (W. Altadena)                          16508.13

C:\Users\nithin.didigam\AppData\Local\Temp\ipykernel_3116\2003800477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['Payout date'] = pd.to_datetime(unique_df['Payout date'], format='%d/%m/%Y')
